# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Feb  2 2023  2:51PM  256557        21          640426   
1  Feb  2 2023  2:45PM  256556        10         9108165   
2  Feb  2 2023  2:40PM  256555        19  ACG-2102492633   
3  Feb  2 2023  2:18PM  256554        10      DNMD-33423   
4  Feb  2 2023  2:18PM  256554        10     Enova-11308   
5  Feb  2 2023  2:17PM  256550        10      0086352490   
6  Feb  2 2023  2:17PM  256550        10      0086352272   
7  Feb  2 2023  2:17PM  256550        10      0086352270   
8  Feb  2 2023  2:17PM  256550        10      0086352275   
9  Feb  2 2023  2:17PM  256550        10      0086352274   

                Customer ShipmentStatus  
0      NBTY Global, Inc.       Released  
1    Digital Brands, LLC       Released  
2  ACG North America LLC       Released  
3            Emerginnova       Released  
4            Emerginnova       Released  
5      ISDIN Corporation       Released  
6      ISDIN Corporation       Released  
7      ISDIN Corporation       Released  
8      ISDIN Corporation       Released  
9      ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
39  256553       Released          3
40  256554       Released          2
41  256555       Released          1
42  256556       Released          1
43  256557       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
256553                NaN        NaN       3.0
256554                NaN        NaN       2.0
256555                NaN        NaN       1.0
256556                NaN        NaN       1.0
256557                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256440                1.0        5.0       0.0
256444                2.0        2.0       6.0
256454                0.0        1.0       0.0
256456                0.0        0.0       1.0
256469                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256440                  1          5         0
256444                  2          2         6
256454                  0          1         0
256456                  0          0         1
256469                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               256440          1          5         0
1               256444          2          2         6
2               256454          0          1         0
3               256456          0          0         1
4               256469          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               256440         1         5         
1               256444         2         2        6
2               256454                   1         
3               256456                            1
4               256469                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0   Feb  2 2023  2:51PM  256557        21      NBTY Global, Inc.
1   Feb  2 2023  2:45PM  256556        10    Digital Brands, LLC
2   Feb  2 2023  2:40PM  256555        19  ACG North America LLC
3   Feb  2 2023  2:18PM  256554        10            Emerginnova
5   Feb  2 2023  2:17PM  256550        10      ISDIN Corporation
28  Feb  2 2023  2:17PM  256549        10      ISDIN Corporation
84  Feb  2 2023  2:15PM  256553        10       Methapharm, Inc.
87  Feb  2 2023  2:13PM  256552        10    Digital Brands, LLC
94  Feb  2 2023  2:11PM  256551        10       Methapharm, Inc.
97  Feb  2 2023  2:04PM  256548        10           Methapharm-G

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Feb  2 2023  2:51PM  256557        21      NBTY Global, Inc.             
1  Feb  2 2023  2:45PM  256556        10    Digital Brands, LLC             
2  Feb  2 2023  2:40PM  256555        19  ACG North America LLC             
3  Feb  2 2023  2:18PM  256554        10            Emerginnova             
4  Feb  2 2023  2:17PM  256550        10      ISDIN Corporation             
5  Feb  2 2023  2:17PM  256549        10      ISDIN Corporation             
6  Feb  2 2023  2:15PM  256553        10       Methapharm, Inc.             
7  Feb  2 2023  2:13PM  256552        10    Digital Brands, LLC             
8  Feb  2 2023  2:11PM  256551        10       Methapharm, Inc.             
9  Feb  2 2023  2:04PM  256548        10           Methapharm-G             

  Executing Released  
0                  1  
1                  1  
2                  1  
3                  2  
4                 23  
5                 56  
6                  3  
7                  7  
8                  3  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Feb  2 2023  2:51PM  256557        21      NBTY Global, Inc.        1   
1  Feb  2 2023  2:45PM  256556        10    Digital Brands, LLC        1   
2  Feb  2 2023  2:40PM  256555        19  ACG North America LLC        1   
3  Feb  2 2023  2:18PM  256554        10            Emerginnova        2   
4  Feb  2 2023  2:17PM  256550        10      ISDIN Corporation       23   
5  Feb  2 2023  2:17PM  256549        10      ISDIN Corporation       56   
6  Feb  2 2023  2:15PM  256553        10       Methapharm, Inc.        3   
7  Feb  2 2023  2:13PM  256552        10    Digital Brands, LLC        7   
8  Feb  2 2023  2:11PM  256551        10       Methapharm, Inc.        3   
9  Feb  2 2023  2:04PM  256548        10           Methapharm-G        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Feb  2 2023  2:51PM  256557        21      NBTY Global, Inc.        1   
1  Feb  2 2023  2:45PM  256556        10    Digital Brands, LLC        1   
2  Feb  2 2023  2:40PM  256555        19  ACG North America LLC        1   
3  Feb  2 2023  2:18PM  256554        10            Emerginnova        2   
4  Feb  2 2023  2:17PM  256550        10      ISDIN Corporation       23   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Feb  2 2023  2:51PM  256557        21      NBTY Global, Inc.       1.0   
1  Feb  2 2023  2:45PM  256556        10    Digital Brands, LLC       1.0   
2  Feb  2 2023  2:40PM  256555        19  ACG North America LLC       1.0   
3  Feb  2 2023  2:18PM  256554        10            Emerginnova       2.0   
4  Feb  2 2023  2:17PM  256550        10      ISDIN Corporation      23.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Feb  2 2023  2:51PM  256557        21      NBTY Global, Inc.       1.0   
1  Feb  2 2023  2:45PM  256556        10    Digital Brands, LLC       1.0   
2  Feb  2 2023  2:40PM  256555        19  ACG North America LLC       1.0   
3  Feb  2 2023  2:18PM  256554        10            Emerginnova       2.0   
4  Feb  2 2023  2:17PM  256550        10      ISDIN Corporation      23.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3078314     104.0        7.0        3.0
12         1026002       3.0        1.0        0.0
15          769410      17.0        0.0        0.0
16          513079       2.0        0.0        0.0
19         1282579      22.0       28.0        9.0
20          256519       1.0        0.0        0.0
21         2308588       7.0        2.0        0.0
22          769566       3.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3078314     104.0        7.0        3.0
1        12  1026002       3.0        1.0        0.0
2        15   769410      17.0        0.0        0.0
3        16   513079       2.0        0.0        0.0
4        19  1282579      22.0       28.0        9.0
5        20   256519       1.0        0.0        0.0
6        21  2308588       7.0        2.0        0.0
7        22   769566       3.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     104.0        7.0        3.0
1        12       3.0        1.0        0.0
2        15      17.0        0.0        0.0
3        16       2.0        0.0        0.0
4        19      22.0       28.0        9.0
5        20       1.0        0.0        0.0
6        21       7.0        2.0        0.0
7        22       3.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  104.0
1        12  Released    3.0
2        15  Released   17.0
3        16  Released    2.0
4        19  Released   22.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    15   16    19   20   21   22
Status                                               
Completed    3.0  0.0   0.0  0.0   9.0  0.0  0.0  0.0
Executing    7.0  1.0   0.0  0.0  28.0  0.0  2.0  0.0
Released   104.0  3.0  17.0  2.0  22.0  1.0  7.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    15   16    19   20   21   22
0          Completed    3.0  0.0   0.0  0.0   9.0  0.0  0.0  0.0
1          Executing    7.0  1.0   0.0  0.0  28.0  0.0  2.0  0.0
2           Released  104.0  3.0  17.0  2.0  22.0  1.0  7.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    15   16    19   20   21   22
0  Completed    3.0  0.0   0.0  0.0   9.0  0.0  0.0  0.0
1  Executing    7.0  1.0   0.0  0.0  28.0  0.0  2.0  0.0
2   Released  104.0  3.0  17.0  2.0  22.0  1.0  7.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()